In [1]:
import pandas as pd
import numpy as np

In [8]:
filename_day_data_nov = 'C:/program2/tensorflow/data/day_data_nov.csv'# 0:00

In [5]:
day_data = np.ones((4320))
day_data = day_data*2
day_data

array([2., 2., 2., ..., 2., 2., 2.])

In [6]:
day_data[144:432] = 1
day_data[1152:1440] = 1
day_data[2160:2448] = 1
day_data[3168:3456] = 1
day_data[4176:4320] = 1
day_data

array([2., 2., 2., ..., 1., 1., 1.])

In [9]:
pd.DataFrame(day_data).to_csv(filename_day_data_nov)

In [10]:
a = pd.read_csv(filename_day_data_nov)
a

,Unnamed: 0,0
0,0,2.0
1,1,2.0
2,2,2.0
3,3,2.0
4,4,2.0
5,5,2.0
6,6,2.0
7,7,2.0
8,8,2.0
9,9,2.0


In [24]:
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Activation, Dropout
from keras.models import load_model

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import time

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def load_data1(filename, seq_len, place):
    set_consumption = pd.read_csv(filename)
    data = set_consumption[str(place)].values
    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index : index + sequence_length])
    result = np.array(result)
    #划分train、test
    row1 = int(0.8 * len(result))
    train = result[:row1, :]
    x_train = train[:, :-1]
    y_train = train[:, -1]

    test = result[row1:, :]
    x_test = test[:, :-1]
    y_test = test[:, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

    return [x_train, y_train, x_test, y_test]

def load_data2(filename_set, filename_tra, seq_len, place):
    set_consumption = pd.read_csv(filename_set)
    set_data = set_consumption[str(place)].values

    traffic = pd.read_csv(filename_tra)
    tra_data = traffic[str(place)].values

    sequence_length = seq_len + 1
    result1 = []
    for index in range(len(set_data) - sequence_length):
        result2 = []
        for j in range(index,index+sequence_length):
            result2.append(set_data[j])
            result2.append(tra_data[j])
        result2 = np.array(result2)
        result1.append(result2)
    result1 = np.array(result1)
    #划分train、test
    row1 = int(0.8 * len(result1))

    train = result1[:row1,:]
    x_train = train[:, :-2]
    y_train = train[:, -2]

    test = result1[row1:, :]
    x_test = test[:, :-2]
    y_test = test[:, -2]

    x_train = np.reshape(x_train, (x_train.shape[0], seq_len, 2))
    x_test = np.reshape(x_test, (x_test.shape[0], seq_len, 2))

    return [x_train, y_train, x_test, y_test]

def load_data3(filename_set, filename_tra, filename_day, seq_len, place):
    set_consumption = pd.read_csv(filename_set)
    set_data = set_consumption[str(place)].values

    traffic = pd.read_csv(filename_tra)
    tra_data = traffic[str(place)].values
    
    day = pd.read_csv(filename_day)
    day_data = day['0'].values
    
    sequence_length = seq_len + 1
    result1 = []
    for index in range(len(set_data) - sequence_length):
        result2 = []
        for j in range(index,index+sequence_length):
            result2.append(set_data[j])
            result2.append(tra_data[j])
            result2.append(day_data[j])
        result2 = np.array(result2)
        result1.append(result2)
    result1 = np.array(result1)
    #划分train、test
    row1 = int(0.8 * len(result1))

    train = result1[:row1,:]
    x_train = train[:, :-3]
    y_train = train[:, -3]

    test = result1[row1:, :]
    x_test = test[:, :-3]
    y_test = test[:, -3]

    x_train = np.reshape(x_train, (x_train.shape[0], seq_len, 3))
    x_test = np.reshape(x_test, (x_test.shape[0], seq_len, 3))

    return [x_train, y_train, x_test, y_test]

def build_model(type):
    model = Sequential()
    if type == 1:
        model.add(LSTM(47, input_shape=(47,1), return_sequences = True))
        model.add(Dropout(0.2))
    elif type == 2:
        model.add(LSTM(47, input_shape=(47,2), return_sequences = True))
        model.add(Dropout(0.2))
    elif type == 3:
        model.add(LSTM(47, input_shape=(47,3), return_sequences = True))
        model.add(Dropout(0.2))
    
    model.add(LSTM(100,return_sequences = False))
    model.add(Dropout(0.2))

    model.add(Dense(output_dim=1))
    model.add(Activation('linear'))

    start = time.time()
    model.compile(loss = 'mse', optimizer = 'rmsprop')
    print('compile time:',time.time() - start)
    return model

def predict(model,data):
    predicted = model.predict(data)
    return predicted

def plot_results_multiple(predicted_data1, predicted_data2, predicted_data3, true_data, place_id):
    x = [i for i in range(len(true_data))]
    y1 = predicted_data1
    y = true_data
    y2 = predicted_data2
    y3 = predicted_data3
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, y, label='True Data')
    ax2 = ax.twinx()
    ax2.plot(x,y2,'r')
    ax1 = ax.twinx()
    ax1.plot(x,y1,'y')
    ax3 = ax.twinx()
    ax3.plot(x,y3,'g')
    plt.savefig('C:/program2/tensorflow/data/lstm_3_data/plot_results_'+str(place_id)+'.jpg')
    #plt.show()

def error(pre1,pre2,pre3,true_data):
    mae_score_pre1 = mean_absolute_error(true_data,pre1)
    mse_score_pre1 = mean_squared_error(true_data,pre1)
    mae_score_pre2 = mean_absolute_error(true_data,pre2)
    mse_score_pre2 = mean_squared_error(true_data,pre2)
    mae_score_pre3 = mean_absolute_error(true_data,pre3)
    mse_score_pre3 = mean_squared_error(true_data,pre3)
    return [mae_score_pre1,mse_score_pre1,mae_score_pre2,mse_score_pre2,mae_score_pre3,mse_score_pre3]

def train(place):
    x_train1, y_train1, x_test1, y_test1 = load_data1(filename_set,47,place)
    x_train2, y_train2, x_test2, y_test2 = load_data2(filename_set,filename_tra,47,place)
    x_train3, y_train3, x_test3, y_test3 = load_data3(filename_set,filename_tra, filename_day, 47, place)
    model1 = build_model(1)
    model1.fit(x_train1,y_train1,batch_size=256,nb_epoch=50,validation_split=0.05)
    model2 = build_model(2)
    model2.fit(x_train2,y_train2,batch_size=256,nb_epoch=50,validation_split=0.05)
    model3 = build_model(3)
    model3.fit(x_train3,y_train3,batch_size=256,nb_epoch=50,validation_split=0.05)
    model1.save('C:/program2/tensorflow/data/lstm_1_data/model1_'+str(place)+'.h5')
    model2.save('C:/program2/tensorflow/data/lstm_2_data/model2_'+str(place)+'.h5')
    model3.save('C:/program2/tensorflow/data/lstm_3_data/model3_'+str(place)+'.h5')
    predict_values1 = predict(model1,x_test1)
    predict_values2 = predict(model2,x_test2)
    predict_values3 = predict(model3,x_test3)
    return predict_values1,predict_values2,predict_values3,y_test1

if __name__=='__main__':
    global filename_tra
    global filename_set
    filename_pla_id = 'C:/program2/tensorflow/data/traffic_corr_id.csv'
    filename_tra = 'C:/program2/tensorflow/data/traffic_de_noising_nov.csv'
    filename_set = 'C:/program2/tensorflow/data/set_consumption_nov.csv'
    #filename_err = 'C:/program2/tensorflow/data/lstm_2_data/error_conclude.csv'
    filename_day = 'C:/program2/tensorflow/data/day_data_nov.csv'

    #model2 = load_model('C:\program2\tensorflow\data\lstm_1_datamodel2_5585.h5')
    #model1 = load_model('model1_5585.h5')
    
    #数据初始化
    error_conclu = pd.DataFrame()
    place_id = pd.read_csv(filename_pla_id)
    place_id = place_id['0'].values
    #traffic_columns = pd.read_csv('C:/program2/tensorflow/data/traffic_columns.csv')
    #traffic_columns = traffic_columns['0'].values
    #set_columns = pd.read_csv('C:/program2/tensorflow/data/set_columns.csv')
    #set_columns = set_columns['0'].values

    #统计每个地点的预测情况
    for i in range(49,len(place_id)):
        #if 'id_'+str(place_id[i]) not in traffic_columns or place_id[i] not in set_columns:
        #    print('the place '+str(place_id[i])+' is missed')
        #    continue
        
        predict_values1,predict_values2,predict_values3,y_test1 = train(place_id[i])
        mae1,mse1,mae2,mse2,mae3,mse3 = error(predict_values1,predict_values2,predict_values3,y_test1)        #能不能合并成一句话 []
        error_conclu[str(place_id[i])] = [mae1,mse1,mae2,mse2,mae3,mse3]
        plot_results_multiple(predict_values1, predict_values2, predict_values3, y_test1, place_id[i])
        print('finish the '+str(place_id[i])+' place')
        error_conclu.to_csv('C:/program2/tensorflow/data/lstm_3_data/err_conclude/de_noising/'+str(place_id[i])+'.csv')
        if i == 49:
            break
    error_conclu.to_csv('C:/program2/tensorflow/data/lstm_3_data/err_conclude/de_noising/high_corr.csv')


D:\software\Anaconda\lib\site-packages\ipykernel_launcher.py:122: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1)`
D:\software\Anaconda\lib\site-packages\keras\models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


compile time: 0.03125309944152832
Train on 3246 samples, validate on 171 samples
Epoch 1/50
3246/3246 [==============================] - 9s 3ms/step - loss: 14.0879 - val_loss: 1.7242
Epoch 2/50
3246/3246 [==============================] - 4s 1ms/step - loss: 2.7393 - val_loss: 1.0778
Epoch 3/50
3246/3246 [==============================] - 4s 1ms/step - loss: 1.7320 - val_loss: 0.7740
Epoch 4/50
3246/3246 [==============================] - 4s 1ms/step - loss: 1.4890 - val_loss: 0.9236
Epoch 5/50
3246/3246 [==============================] - 4s 1ms/step - loss: 1.2029 - val_loss: 0.1400
Epoch 6/50
3246/3246 [==============================] - 4s 1ms/step - loss: 0.9467 - val_loss: 0.9671
Epoch 7/50
3246/3246 [==============================] - 4s 1ms/step - loss: 0.8893 - val_loss: 0.0982
Epoch 8/50
3246/3246 [==============================] - 4s 1ms/step - loss: 0.9151 - val_loss: 0.2917
Epoch 9/50
3246/3246 [==============================] - 4s 1ms/step - loss: 0.7557 - val_loss: 0.1206


3246/3246 [==============================] - 4s 1ms/step - loss: 1.2012 - val_loss: 1.4947
Epoch 30/50
3246/3246 [==============================] - 4s 1ms/step - loss: 1.1546 - val_loss: 0.4533
Epoch 31/50
3246/3246 [==============================] - 4s 1ms/step - loss: 1.1246 - val_loss: 0.7683
Epoch 32/50
3246/3246 [==============================] - 4s 1ms/step - loss: 1.1265 - val_loss: 0.4165
Epoch 33/50
3246/3246 [==============================] - 4s 1ms/step - loss: 0.9893 - val_loss: 0.3598
Epoch 34/50
3246/3246 [==============================] - 4s 1ms/step - loss: 1.1654 - val_loss: 0.4754
Epoch 35/50
3246/3246 [==============================] - 4s 1ms/step - loss: 1.1455 - val_loss: 0.4372
Epoch 36/50
3246/3246 [==============================] - 4s 1ms/step - loss: 0.8271 - val_loss: 0.9454
Epoch 37/50
3246/3246 [==============================] - 4s 1ms/step - loss: 1.1565 - val_loss: 0.6916
Epoch 38/50
3246/3246 [==============================] - 4s 1ms/step - loss: 1.1303 -

In [12]:
a = pd.read_csv('C:/program2/tensorflow/data/traffic_corr_id.csv')
a = a['0'].values
a

array([  277,  2502,  2737,  2738,  2853,  2855,  2971,  2972,  2973,
        3089,  3090,  3191,  3192,  3322,  4031,  4148,  4265,  4266,
        4384,  4385,  4500,  4501,  4616,  4617,  4618,  4619,  4731,
        4733,  4734,  4735,  4736,  4848,  4849,  4850,  4851,  4852,
        4966,  4967,  4968,  4970,  5083,  5084,  5085,  5087,  5200,
        5201,  5202,  5203,  5204,  5205,  5206,  5207,  5317,  5318,
        5319,  5320,  5323,  5324,  5433,  5434,  5435,  5436,  5437,
        5438,  5439,  5441,  5550,  5551,  5552,  5553,  5554,  5667,
        5668,  5669,  5676,  5783,  5784,  6019,  6136,  6137,   276,
         393,   978,   979,   982,  1096,  1097,  1331,  1566,  1683,
        1684,  1800,  1915,  1916,  1917,  2022,  2032,  2033,  2034,
        2139,  2148,  2149,  2150,  2151,  2256,  2258,  2264,  2265,
        2266,  2267,  2268,  2373,  2381,  2382,  2383,  2384,  2385,
        2491,  2501,  2607,  2608,  2619,  2623,  2725,  2736,  2845,
        2960,  2961,

In [23]:
ind = list(a).index(5205)
ind

49